### 개발환경 설정 및 데이터 가져오기

In [ ]:
# For Colab Use------------------------------------
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Package loading ===============================================
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import matplotlib.font_manager as fm
import seaborn as sns
# import missingno as msno
# nltk.download("all")

In [ ]:
# Import text clearning libraries
import re # for regular expression operations
from bs4 import BeautifulSoup
# 자연어처리를 위한 라이브러리, NLTK importing
import nltk
from wordcloud import WordCloud, STOPWORDS , ImageColorGenerator
nltk.download("all")
import string

#Frequent words
from collections import Counter

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# column width 조정
pd.set_option('display.max_colwidth', 200)

### 데이터로드

In [ ]:
# 데이터 가져오기
df = pd.read_csv("/content/drive/MyDrive/PropertyGuru/PreparedCSV/All_until_10words_220120.csv", index_col=0)

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 258702 entries, 0 to 258702
Data columns (total 35 columns):
 #   Column                                  Non-Null Count   Dtype  
---  ------                                  --------------   -----  
 0   ListingID                               258702 non-null  float64
 1   created_at                              258701 non-null  object 
 2   listed_date                             258701 non-null  object 
 3   description                             258702 non-null  object 
 4   Floorsize                               258702 non-null  float64
 5   Floor                                   211998 non-null  object 
 6   House_type                              258702 non-null  object 
 7   Lease_type                              258702 non-null  object 
 8   region                                  179479 non-null  object 
 9   years_from_TOP                          258701 non-null  float64
 10  PSF_float                               2587

### 데이터 타입 형변환

#### 정수변환

In [ ]:
# 정수로 변환하고자 하는 변수 정수로 변환(결측치 없는 경우만 작동됨)
to_int_needed = ['price', 'TOP'] 
df[to_int_needed] = df[to_int_needed].applymap(np.int64)

#### 문자변환

In [ ]:
# 문자로 변환하고자 하는 변수: ListingID
df['ListingID'] = df['ListingID'].astype(int).astype(str)

### 추가적 변수 생성

#### Month

In [ ]:
# 시간변수로 데이터타입 변경 object --> datetime64
df.listed_date = pd.to_datetime(df.listed_date)

In [ ]:
df.info()

In [ ]:
df['Month'] = pd.DatetimeIndex(df['listed_date']).month

In [ ]:
df[['listed_date','Month']].value_counts() # 확인

listed_date  Month
2019-10-26   10.0     10199
2019-10-20   10.0      8579
2019-10-25   10.0      7797
2020-09-25   9.0       7552
2020-12-04   12.0      6830
                      ...  
2020-12-11   12.0         5
2020-08-13   8.0          5
2020-07-14   7.0          2
2020-11-11   11.0         1
2020-05-09   5.0          1
Length: 149, dtype: int64

#### 팬데믹Phase 시간대 구분 변수 생성


In [ ]:
# 방법 1 
# 일단 phase라는 새로운 변수를 생성
df['phase'] = 'None'

In [ ]:
df[['listed_date', 'phase']]

,listed_date,phase
0,2019-10-26,None
1,2019-10-26,None
2,2019-10-26,None
3,2019-10-26,None
4,2019-10-26,None
...,...,...
258698,2021-07-21,None
258699,2021-07-16,None
258700,2021-07-16,None
258701,2021-07-16,None


#### 호섭님 코드 reference

In [ ]:
# 시간대 구분함수 만들기
def cut(x):
    if x <= '2020-01-23':
        return  'before covid'
    if '2020-04-07' <= x <= '2020-06-01':
        return 'circuit breaker'
    if '2020-06-02' <= x <= '2020-06-18':
        return 'phase1'
    if '2020-06-19' <= x <= '2020-12-27':
        return 'phase2'
    if '2020-12-28' <= x <= '2021-05-07':
        return 'phase3'
    if '2021-05-08' <= x <= '2021-05-15':
        return 'phase3(Heightened Alert)'
    if '2021-05-16' <= x <= '2021-06-13':
        return 'phase2(Heightened Alert)'
    if '2021-06-14' <= x <= '2021-06-20':
        return 'phase3(Heightened Alert) with Stage 1'
    if '2021-06-21' <= x <= '2021-07-11':
        return 'phase3(Heightened Alert) with Stage 2'
    if '2021-07-12' <= x <= '2021-07-21':
        return 'phase3(Heightened Alert)'    
    if '2021-07-22' <= x <= '2021-08-09':
        return 'phase2(Heightened Alert)'  

In [ ]:
df['phase'] = df['listed_date'].apply(lambda x : cut(x))
df[['phase', 'listed_date']].sample(20)

TypeError: ignored

#### phase 구분

In [ ]:
# phase cuts - 서상준 선생님 advice
import datetime
df.loc[(df['listed_date'] <=datetime.datetime(2020,1,23)), 'phase'] = 'before covid'
df.loc[(df['listed_date'] >=datetime.datetime(2020,4,7)) & (df['listed_date'] <=datetime.datetime(2020,6,1) ), 'phase'] = 'Circuit breaker lockdown'
df.loc[(df['listed_date'] >=datetime.datetime(2020,6,2)) & (df['listed_date'] <=datetime.datetime(2020,6,18) ), 'phase'] = 'phase1'
df.loc[(df['listed_date'] >=datetime.datetime(2020,6,19)) & (df['listed_date'] <=datetime.datetime(2020,12,27) ), 'phase'] = 'phase2'
df.loc[(df['listed_date'] >=datetime.datetime(2020,12,28)) & (df['listed_date'] <=datetime.datetime(2021,5,7) ), 'phase'] = 'phase3'
df.loc[(df['listed_date'] >=datetime.datetime(2021,5,8)) & (df['listed_date'] <=datetime.datetime(2021,5,15) ), 'phase'] = 'phase3(Heightened Alert)'
df.loc[(df['listed_date'] >=datetime.datetime(2021,5,16)) & (df['listed_date'] <=datetime.datetime(2021,6,13) ), 'phase'] = 'phase2(Heightened Alert)'
df.loc[(df['listed_date'] >=datetime.datetime(2021,6,14)) & (df['listed_date'] <=datetime.datetime(2021,6,20) ), 'phase'] = 'phase3(Heightened Alert) with Stage1'
df.loc[(df['listed_date'] >=datetime.datetime(2021,6,21)) & (df['listed_date'] <=datetime.datetime(2021,7,11) ), 'phase'] = 'phase3(Heightened Alert) with Stage2'
df.loc[(df['listed_date'] >=datetime.datetime(2021,7,12)) & (df['listed_date'] <=datetime.datetime(2021,7,21) ), 'phase'] = 'phase3(Heightened Alert)'
df.loc[(df['listed_date'] >=datetime.datetime(2021,7,22)) & (df['listed_date'] <=datetime.datetime(2021,8,9) ), 'phase'] = 'phase2(Heightened Alert)'

In [ ]:
df[['listed_date','phase']].sample(20)

,listed_date,phase
118960,2020-11-01,phase2
248005,2021-07-21,phase3(Heightened Alert)
171264,2021-01-15,phase3
85263,2020-09-26,phase2
83095,2020-09-26,phase2
45833,2019-10-26,before covid
228054,2021-05-31,phase2(Heightened Alert)
224343,2021-03-27,phase3
183404,2021-01-11,phase3
214031,2021-03-27,phase3


### 컬럼 순서 변경

In [ ]:
df.columns # 현재 순서

Index(['ListingID', 'created_at', 'listed_date', 'description', 'Floorsize',
       'Floor', 'House_type', 'Lease_type', 'region', 'years_from_TOP',
       'PSF_float', 'Tenanted', 'Total Unit(s)', 'Furnishing', 'listed_year',
       'pandemic', 'address.locality', 'latitude', 'longitude', 'rooms',
       'bathrooms', 'floor_size', 'price', 'days_past', 'Condo', 'HDB', 'TOP',
       'Lease', 'Free', 'Developer', 'image_urls', 'URL',
       'development_properties.Completion Year', 'properties.url', 'descrip10',
       'Month', 'phase'],
      dtype='object')

In [ ]:
# 바꾸고 싶은 순서대로 순서 변경
df = df[['ListingID', 'created_at','listed_date','description','descrip10','Floorsize', 'Floor',
         'House_type','Lease_type', 'region','years_from_TOP','PSF_float','Tenanted','Total Unit(s)', 'Furnishing',
         'listed_year', 'Month','phase', 'pandemic', 
         'address.locality', 'latitude', 'longitude', 'rooms', 'bathrooms', 'floor_size', 'price',
         'days_past', 'Condo', 'HDB', 'TOP', 'Lease', 'Free', 'Developer', 
         'image_urls','URL', 'development_properties.Completion Year','properties.url']]

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 258702 entries, 0 to 258702
Data columns (total 37 columns):
 #   Column                                  Non-Null Count   Dtype         
---  ------                                  --------------   -----         
 0   ListingID                               258702 non-null  object        
 1   created_at                              258701 non-null  object        
 2   listed_date                             258701 non-null  datetime64[ns]
 3   description                             258702 non-null  object        
 4   descrip10                               258702 non-null  object        
 5   Floorsize                               258702 non-null  float64       
 6   Floor                                   211998 non-null  object        
 7   House_type                              258702 non-null  object        
 8   Lease_type                              258702 non-null  object        
 9   region                               

# 시점별 리스팅 갯수 파악

### 게시물별 고유ID 생성

In [ ]:
# 결측 채우기
df['Floor'] = df['Floor'].fillna('missing')

In [ ]:
df['TOP'].isna().sum()

0

In [ ]:
# 전체 데이터를 description과 게시날짜 기준으로 정렬
df.sort_values(by = ['descrip10', 'listed_date'], ascending = True).head()
# 문장과 게시날짜 기준으로 정렬해서 봤더니, 문장과 게시날짜가 동일해도 Floor(층수)가 다른 경우 존재. 

In [ ]:
# 따라서 Floor,latitude, longitude, descrip10, TOP로 매물고유번호를 생성하여 정렬하기로 함.
df['property_UID'] = df['Floor'] + df['latitude'].astype(str) + df['longitude'].astype(str) + df['descrip10']+ df['TOP'].astype(str)

In [ ]:
# 전체 데이터를 property_UID와 listed_date를 기준으로 정렬
df = df.sort_values(by = ['property_UID', 'listed_date'], ascending = True)

In [ ]:
# 컬럼순서 변경
df = df[['property_UID', 'listed_date','description','descrip10','Floorsize', 'Floor',
         'House_type','Lease_type', 'region','ListingID', 'created_at','years_from_TOP','PSF_float','Tenanted','Total Unit(s)', 'Furnishing',
         'listed_year', 'Month','phase', 'pandemic', 
         'address.locality', 'latitude', 'longitude', 'rooms', 'bathrooms', 'floor_size', 'price',
         'days_past', 'Condo', 'HDB', 'TOP', 'Lease', 'Free', 'Developer', 
         'image_urls','URL', 'development_properties.Completion Year','properties.url']]

In [ ]:
len(df)

258702

### new / reupload 포스팅 구분변수

In [ ]:
## Property_UID duplicates 중에 첫행 뽑아서 (keep='first) 
df_UID_first = df.drop_duplicates(['property_UID'], keep='first', ignore_index=False)
len(df_UID_first)

138849

In [ ]:
# 각 property(주택)별로 처음 나온 새로운 게시글의 인덱스 추출, new_list 변수에 저장하고 조회
list_of_index = [item for item in df_UID_first.index]
# 5개만 예시로 보면
list_of_index[:5] 

[44535, 27452, 100315, 239036, 93284]

In [ ]:
# 데이터프레임 인덱스를 복사해서 컬럼으로 만듦
df['index'] = df.index

In [ ]:
df.head()

### 새로운게시물더미: new 변수 생성

In [ ]:
# 전체 df에서 그 각각 행의 index가 new_list의 원소이면 new=1, 아니면 new=0
df['new'] = np.where(df['index'].isin(list_of_index), 1, 0)
df[['property_UID','listed_date', 'new', 'descrip10']].head(10)

,property_UID,listed_date,new,descrip10
44535,Ground Floor1.239435103.836842beautiful grind floor corner 4 bedroom apartment full sea view2012,2019-10-06,1,beautiful grind floor corner 4 bedroom apartment full sea view
162313,Ground Floor1.239435103.836842beautiful grind floor corner 4 bedroom apartment full sea view2012,2020-11-21,0,beautiful grind floor corner 4 bedroom apartment full sea view
171631,Ground Floor1.239435103.836842beautiful grind floor corner 4 bedroom apartment full sea view2012,2021-01-15,0,beautiful grind floor corner 4 bedroom apartment full sea view
203469,Ground Floor1.239435103.836842beautiful grind floor corner 4 bedroom apartment full sea view2012,2021-02-18,0,beautiful grind floor corner 4 bedroom apartment full sea view
236171,Ground Floor1.239435103.836842beautiful grind floor corner 4 bedroom apartment full sea view2012,2021-05-28,0,beautiful grind floor corner 4 bedroom apartment full sea view
27452,Ground Floor1.239435103.836842grind floor sea view experience breath take view seascape sentosa2012,2019-10-20,1,grind floor sea view experience breath take view seascape sentosa
100315,Ground Floor1.239435103.836842luxurious live patio overlook sea grind floor unit 3 bedrooms2012,2020-09-18,1,luxurious live patio overlook sea grind floor unit 3 bedrooms
130768,Ground Floor1.239435103.836842luxurious live patio overlook sea grind floor unit 3 bedrooms2012,2020-11-08,0,luxurious live patio overlook sea grind floor unit 3 bedrooms
167870,Ground Floor1.239435103.836842luxurious live patio overlook sea grind floor unit 3 bedrooms2012,2020-11-14,0,luxurious live patio overlook sea grind floor unit 3 bedrooms
190425,Ground Floor1.239435103.836842luxurious live patio overlook sea grind floor unit 3 bedrooms2012,2021-01-11,0,luxurious live patio overlook sea grind floor unit 3 bedrooms


### reupload 여부

In [ ]:
# reupload변수 생성
# 전체 df에서 그 각각 행의 index가 new_list의 원소이면 new, 아니면 reupload
df['reupload'] = np.where(df['index'].isin(list_of_index), 'new', 'reupload')
df[['property_UID', 'listed_date','reupload','new','descrip10']] 

,property_UID,listed_date,reupload,new,descrip10
44535,Ground Floor1.239435103.836842beautiful grind floor corner 4 bedroom apartment full sea view2012,2019-10-06,new,1,beautiful grind floor corner 4 bedroom apartment full sea view
162313,Ground Floor1.239435103.836842beautiful grind floor corner 4 bedroom apartment full sea view2012,2020-11-21,reupload,0,beautiful grind floor corner 4 bedroom apartment full sea view
171631,Ground Floor1.239435103.836842beautiful grind floor corner 4 bedroom apartment full sea view2012,2021-01-15,reupload,0,beautiful grind floor corner 4 bedroom apartment full sea view
203469,Ground Floor1.239435103.836842beautiful grind floor corner 4 bedroom apartment full sea view2012,2021-02-18,reupload,0,beautiful grind floor corner 4 bedroom apartment full sea view
236171,Ground Floor1.239435103.836842beautiful grind floor corner 4 bedroom apartment full sea view2012,2021-05-28,reupload,0,beautiful grind floor corner 4 bedroom apartment full sea view
...,...,...,...,...,...
188192,missing1.457707103.8375049top soon lowest psf agent fee agent fee payable hide2021,2021-01-17,reupload,0,top soon lowest psf agent fee agent fee payable hide
243268,missing1.457707103.8375049top soon lowest psf agent fee agent fee payable hide2021,2021-05-09,reupload,0,top soon lowest psf agent fee agent fee payable hide
197891,missing1.457707103.8375049top soon price reduce kandis residence 99 years leasehold condominium2021,2020-12-28,new,1,top soon price reduce kandis residence 99 years leasehold condominium
173660,missing1.457707103.8375049view actual special top discount call buy direct developer best2021,2021-01-16,new,1,view actual special top discount call buy direct developer best


In [ ]:
df.columns

Index(['property_UID', 'listed_date', 'description', 'descrip10', 'Floorsize',
       'Floor', 'House_type', 'Lease_type', 'region', 'ListingID',
       'created_at', 'years_from_TOP', 'PSF_float', 'Tenanted',
       'Total Unit(s)', 'Furnishing', 'listed_year', 'Month', 'phase',
       'pandemic', 'address.locality', 'latitude', 'longitude', 'rooms',
       'bathrooms', 'floor_size', 'price', 'days_past', 'Condo', 'HDB', 'TOP',
       'Lease', 'Free', 'Developer', 'image_urls', 'URL',
       'development_properties.Completion Year', 'properties.url', 'index',
       'new', 'reupload'],
      dtype='object')

In [ ]:
# 컬럼순서 변경
df = df[['index','property_UID', 'listed_date','reupload','new','descrip10','description',
         'Floorsize', 'Floor','House_type','Lease_type', 'region',
         'ListingID', 'created_at','years_from_TOP','PSF_float','Tenanted','Total Unit(s)', 'Furnishing',
         'listed_year', 'Month','phase', 'pandemic', 
         'address.locality', 'latitude', 'longitude', 'rooms', 'bathrooms', 'floor_size', 'price',
         'days_past', 'Condo', 'HDB', 'TOP', 'Lease', 'Free', 'Developer', 
         'image_urls','URL', 'development_properties.Completion Year','properties.url']]

In [ ]:
df.to_csv("/content/drive/MyDrive/PropertyGuru/PreparedCSV/EDA_ready_with_property_UID.csv")